# Projektna naloga

Za projektno nalogo sem se odločila, da bom analizirala podatke o rakavih obolenjih. Glavni cilj te projektne naloge je, da temeljito analizira statistične podatke posameznega tipa raka ter te podatke ustrezno in razumljivo predstavi.

Najprej uvozimo knjižnjice, ki jih bomo potrebovali. 

In [129]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os

Za tem definiramo spletno stran iz katere jemljemo podatke in jo z BeautifulSoup uvozimo v spremenljivko `bs_spletna_stran`.

In [130]:
link = 'https://seer.cancer.gov'
link_podrocja = link + '/statfacts/'

spletna_stran = urlopen(link_podrocja).read().decode('UTF-8')
bs_spletna_stran = BeautifulSoup(spletna_stran, 'html.parser')

Na spletni strani pogledamo kje se nahajajo podatki, ki jih želimo. Ugotovimo da so zajeti v razredu `alphaList`. Z metodo `.find()` poiščemo vse podatke v `alphaList` in jih shranimo v spremenljivko seznam_podrocij.

In [131]:
seznam_podrocij = bs_spletna_stran.find('div', attrs={'class': 'alphaList'})

Za boljši pregled ustvarimo prazen slovar `slovar_podrocij` v katerega bomo shranili kot ključ tip raka in pod vrednosti povezave za vsak tip posebaj.

In [132]:
slovar_podrocij = {}

Na spletni strani so nekateri podatki neuporabni in jih zato lahko izpustimo. 

In [133]:
nepotrebne_teme = [
    'Female Breast Subtypes', 'Chronic Lymphocytic Leukemia/Small Lymphocytic Lymphoma (CLL/SLL)',
    'Diffuse Large B-Cell Lymphoma (DLBCL)', 'Follicular Lymphoma', 'Lip'
]

for podrocje in seznam_podrocij.find_all('a'):
    if podrocje.get_text() in nepotrebne_teme:
        continue
    slovar_podrocij[podrocje.get_text()] = podrocje.get('href')

Za hitrejše in bolj pregledno kodo, bomo napisali funkcijo, ki bom iz spletne strani pobrala potrebne podatke. Ker so podatki shranjeni pod različne značke, bomo znotraj funkcije napisali pomožne funkcije. V vsaki izmed teh funkcij najprej po znački poiščemo podatke in jih dobimo z `.get_text()`. Vse dobljene podatke shranimo v nov prazen slovar `podatki`, kjer ključe določimo z tipom podatka, za vrednosti pa uzamemo poiskani podatek.

In [134]:
def pridobi_podatke(povezava):
    podatki = {}

    def novi_in_smrtni_primeri(vrsta, podatek):
        for p in podatek.find_all('p'):
            naslov = p.find('span').get_text()
            vrednost = p.find('span').find_next().get_text().replace(',', '')
            estimated = re.search(r'^Estimated', naslov)

            if estimated:
                podatki['prb_' + vrsta] = vrednost
            else:
                podatki['delez_' + vrsta] = vrednost

    def ostali_podatki(kljuc, podatek):
        tag = 'td'
        if 'povp_starost' in kljuc:
            tag = 'strong'
        podatek_spol = podatek.find(tag).get_text().replace(',', '')
        podatki[kljuc] = podatek_spol

    def spletna_povezava(oznaka, tip, sez):
        podatki_za_predelavo = spletna_podrocje.find(oznaka, attrs={tip: sez})
        return podatki_za_predelavo

    link_tema = link + povezava
    spletna_podrocje = BeautifulSoup(urlopen(link_tema).read().decode('UTF-8'), 'html.parser')

    # novi primeri
    if spletna_povezava('div', 'class', ['new', 'glanceBox']):
        novi_in_smrtni_primeri('novi', spletna_povezava('div', 'class', ['new', 'glanceBox']))

    # smrtni primeri
    if spletna_povezava('div', 'class', ['death']):
        novi_in_smrtni_primeri('smrtni', spletna_povezava('div', 'class', ['death']))

    # podatki o relativnem prezivetje
    if spletna_povezava('div', 'class', ['col-lg-4', 'offset-lg-1']):
        for div in spletna_povezava('div', 'class', ['col-lg-4', 'offset-lg-1']).find_all('div'):
            podatki_prezivetje = div.find('strong').get_text().replace('%', '')
            podatki['5_letno_realtivno_prezivetje_stevilo'] = podatki_prezivetje

    # podatki o novih primerih moski
    if spletna_povezava('table', 'id', ['scrapeTable_04']):
        ostali_podatki('vse_rase_moski', spletna_povezava('table', 'id', ['scrapeTable_04']))

    # podatki o novih primerih zenske
    if spletna_povezava('table', 'id', ['scrapeTable_05']):
        ostali_podatki('vse_rase_zenske', spletna_povezava('table', 'id', ['scrapeTable_05']))

    # podatki o povprečni starosti za nove primere
    if spletna_povezava('div', 'class', ['statSurv']):
        ostali_podatki('povp_starost', spletna_povezava('div', 'class', ['statSurv']))

    # podatki o smrtnih primerih moski
    if spletna_povezava('table', 'id', ['scrapeTable_07']):
        ostali_podatki('smrtni_vse_rase_moski', spletna_povezava('table', 'id', ['scrapeTable_07']))

    # podatki o smrtnih primerih zenske
    if spletna_povezava('table', 'id', ['scrapeTable_08']):
        ostali_podatki('smrtni_vse_rase_zenske', spletna_povezava('table', 'id', ['scrapeTable_08']))

    # podatki o povprecni starosti smrti
    if spletna_povezava('div', 'class', ['col-lg-4', 'offset-lg-1']):
        ostali_podatki('povp_starost_smrti', spletna_povezava('div', 'class', ['statDie']))

    return podatki

Za boljšo preglednost, podatke v slovarju `podatki` spravimo v prvotni slovar, kjer so ključi tipi raka. Tako imamo v slovarju za vsak tip raka posebaj zbrane vse podatke.

In [135]:
for k, v in slovar_podrocij.items():
    slovar_podrocij[k] = pridobi_podatke(v)

Zdaj ustvarimo mapo, kamor bomo shranjevali izrisane grafe. To nam omogoča boljšo preglednost.

In [136]:
mapa_grafi = 'grafi'
mapa_obstaja = os.path.exists(mapa_grafi)
if not mapa_obstaja:
    os.makedirs(mapa_grafi)
mapa_grafi += '/'

Ker se iz slovarja zelo težko znajdemo bomo raje naredili csv datoteko, kjer bodo podatki bolj razvidni. Zopet bomo definiral funkcijo in sicer `ustvari_csv`, kjer bomo najprej odpli datoteko, nato ustvarimo glavo csv datoteko, to so imena stolpcev naše tabele. Za tem bomo z zanko `for` šli čez vse ključe in vrednosti v slovarju. Tu si pomagamo z metodo `.get()` z katero dobimo željeno vrednost iz slovarja. 

In [137]:
csv_ime_datoteke = 'output.csv'


def ustvari_csv(ime_datoteke):
    with open(ime_datoteke, mode='w', newline='') as file:
        writer = csv.writer(file)

        header = ['Type Of Cancer', 'Estimated New Cases in 2023', '(%) of All New Cancer Cases',
                  'Estimated Deaths in 2023',
                  '(%) of All Cancer Deaths', '5-Year Relative Survival (%)',
                  'New Cases Male (%)', 'New Cases Female (%)', 'Median Age At Diagnosis',
                  'Death Cases Male (%)', 'Death Cases Female (%)', 'Median Age At Death']
        writer.writerow(header)

        for name, values in slovar_podrocij.items():
            prb_novi = values.get('prb_novi')
            delez_novi = values.get('delez_novi')
            prb_smrti = values.get('prb_smrtni')
            delez_smrti = values.get('delez_smrtni')
            relativno_prezivetje_stevilo = values.get('5_letno_realtivno_prezivetje_stevilo')
            vse_rase_m = values.get('vse_rase_moski')
            vse_rase_z = values.get('vse_rase_zenske')
            povp_starost = values.get('povp_starost')
            smrtni_vse_rase_m = values.get('smrtni_vse_rase_moski')
            smrtni_vse_rase_z = values.get('smrtni_vse_rase_zenske')
            povp_starost_smrti = values.get('povp_starost_smrti')

            row = [name, prb_novi, delez_novi, prb_smrti, delez_smrti, relativno_prezivetje_stevilo,
                   vse_rase_m, vse_rase_z, povp_starost, smrtni_vse_rase_m,
                   smrtni_vse_rase_z, povp_starost_smrti]
            writer.writerow(row)


Zadnja stvar, ki jo moramo narediti je ustvariti csv datoteko. To naredimo tako, da poženemo našo funkcijo `ustvari_csv`.

In [138]:
ustvari_csv(csv_ime_datoteke)

Za anlizo podatkov uporabimo knjižnjico `pandas`. Z `pd.read_csv()` preberemo ustvarjeno csv datoteko. Opazimo, da nekateri podatki mankajo oziroma imajo podatek v obliki teksta. Za lažjo obdelavo podatkov te nadomestili z vrednostjo 0 in zamenjamo tip podatka v numeričnega.

In [139]:
def predelaj_csv_podatke(ime_datoteke, zapolni_privzete=False):
    predelani_podatki = pd.read_csv(ime_datoteke, encoding='cp1252', index_col='Type Of Cancer')
    if zapolni_privzete:
        predelani_podatki = predelani_podatki.fillna(0)
        predelani_podatki.replace('Sex-specific cancer type', 0, inplace=True)

    # številčne podatke spremenimo v numerične za lažjo obdelavo
    stolpci = ['Estimated New Cases in 2023', 'Estimated Deaths in 2023']
    predelani_podatki[stolpci] = predelani_podatki[stolpci].applymap(pd.to_numeric, errors='coerce').astype(int)

    return predelani_podatki

Uporabimo funkcijo in shranimo pod spremenljivko.

In [140]:
vsi_podatki = predelaj_csv_podatke(csv_ime_datoteke, zapolni_privzete=True)

Zdaj ko so podatki v ustrezni obliki jih lahko filtriramo. Najprej bomo prikazali pričakovano število novih primerov. Te uredimo po velikosti z `.sort_values()`.

In [141]:
pricakovani_novi_primeri = vsi_podatki[['Estimated New Cases in 2023']].sort_values('Estimated New Cases in 2023',
                                                                               ascending=False)

Odstranimo vse vrstice, kjer je 0.

In [142]:
pricakovani_novi_primeri = pricakovani_novi_primeri[pricakovani_novi_primeri['Estimated New Cases in 2023'] != 0]

V tabeli so tako prikazani podatki za pričakovano število novih primerov za vsak tip raka posebaj. Ker smo vrednosti uredili po velikosti, lahko iz tabele preberemo kateri tip raka je najpogostejši torej rak dojk.

In [143]:
print(pricakovani_novi_primeri)

                                    Estimated New Cases in 2023
Type Of Cancer                                                 
Breast (Female)                                          297790
Prostate                                                 288300
Lung and Bronchus                                        238340
Colon and Rectum                                         153020
Melanoma of the Skin                                      97610
Bladder                                                   82290
Kidney and Renal Pelvis                                   81800
Non-Hodgkin Lymphoma                                      80550
Uterus                                                    66200
Pancreas                                                  64050
Leukemia                                                  59610
Oral Cavity and Pharynx                                   54540
Thyroid                                                   43720
Liver and Intrahepatic Bile Duct        

Enako naredimo še za smrtne primere.

In [144]:
pricakovani_smrtni_primeri = vsi_podatki[['Estimated Deaths in 2023']].sort_values('Estimated Deaths in 2023',
                                                                              ascending=False)
pricakovani_smrtni_primeri = pricakovani_smrtni_primeri[pricakovani_smrtni_primeri['Estimated Deaths in 2023'] != 0]

Naslednja tabela je zelo podobna prejšnji, le da so tu prikazani podatki za smrtne primere. 

In [145]:
print(pricakovani_smrtni_primeri)

                                    Estimated Deaths in 2023
Type Of Cancer                                              
Lung and Bronchus                                     127070
Colon and Rectum                                       52550
Pancreas                                               50550
Breast (Female)                                        43170
Prostate                                               34700
Liver and Intrahepatic Bile Duct                       29380
Leukemia                                               23710
Non-Hodgkin Lymphoma                                   20180
Brain and Other Nervous System                         18990
Bladder                                                16710
Esophagus                                              16120
Kidney and Renal Pelvis                                14890
Ovary                                                  13270
Uterus                                                 13030
Myeloma                 

Poleg podatkov o novih in smrtnih primerih, je dobro vedeti pri katerem spolu je določen tip pogostejši. V naslednjih tabelah so prikazani podatki o odkritju novih primerov pri moških in ženskah ter povprečna starost pri kateri odkrijejo specifičen tip raka. Ponekod se pojavi `\`, to zato ker so določeni tipi specifični za določen spol recimo rak materničnega vrata.

Ta tabela prikazuje podatke o novih primerih odkritja raka pri moških v odstotkih.

In [146]:
kdo_dobi_tega_raka_m = vsi_podatki[['New Cases Male (%)']].replace(0, '/')
print(kdo_dobi_tega_raka_m)

                                                   New Cases Male (%)
Type Of Cancer                                                       
Anus                                                              1.6
Bladder                                                          31.7
Bone and Joint                                                    1.2
Brain and Other Nervous System                                    7.4
Breast (Female)                                                     /
Cervix Uteri                                                        /
Childhood Brain and Other Nervous System (Ages ...                3.2
Childhood Leukemia (Ages 0–19)                                    5.4
Colon and Rectum                                                 42.1
Esophagus                                                         7.2
Kidney and Renal Pelvis                                          23.5
Larynx                                                            4.7
Liver and Intrahepat

Ta tabela prikazuje podatke o novih primerih odkritja raka pri ženskah v odstotkih.

In [147]:
kdo_dobi_tega_raka_z = vsi_podatki[['New Cases Female (%)']].replace(0, '/')
print(kdo_dobi_tega_raka_z)

                                                   New Cases Female (%)
Type Of Cancer                                                         
Anus                                                                2.3
Bladder                                                             7.8
Bone and Joint                                                      0.9
Brain and Other Nervous System                                      5.2
Breast (Female)                                                   126.9
Cervix Uteri                                                        7.7
Childhood Brain and Other Nervous System (Ages ...                  2.9
Childhood Leukemia (Ages 0–19)                                      4.3
Colon and Rectum                                                   32.0
Esophagus                                                           1.7
Kidney and Renal Pelvis                                            11.7
Larynx                                                          

Ta tabela prikazuje podatke o povprečni starosti pri kateri odkrijejo določen tip raka.

In [148]:
povp_starost_diagnoze = vsi_podatki[['Median Age At Diagnosis']].replace(0, '/')
print(povp_starost_diagnoze)

                                                    Median Age At Diagnosis
Type Of Cancer                                                             
Anus                                                                     63
Bladder                                                                  73
Bone and Joint                                                           46
Brain and Other Nervous System                                           60
Breast (Female)                                                          63
Cervix Uteri                                                             50
Childhood Brain and Other Nervous System (Ages ...                        8
Childhood Leukemia (Ages 0–19)                                            6
Colon and Rectum                                                         66
Esophagus                                                                68
Kidney and Renal Pelvis                                                  65
Larynx      

Seveda nas poleg odkritih novih primerov zanima še kako je z smrtnostjo, zato enako naredimo še za smrtne primere.

Ta tabela prikazuje podatke o smrtnih primerih odkritja raka pri moških v odstotkih.

In [149]:
kdo_umre_od_raka_m = vsi_podatki[['Death Cases Male (%)']].replace(0, '/')
print(kdo_umre_od_raka_m)

                                                   Death Cases Male (%)
Type Of Cancer                                                         
Anus                                                                0.3
Bladder                                                             7.1
Bone and Joint                                                      0.6
Brain and Other Nervous System                                      5.4
Breast (Female)                                                       /
Cervix Uteri                                                          /
Childhood Brain and Other Nervous System (Ages ...                  0.7
Childhood Leukemia (Ages 0–19)                                      0.6
Colon and Rectum                                                   15.7
Esophagus                                                           6.7
Kidney and Renal Pelvis                                             5.1
Larynx                                                          

Ta tabela prikazuje podatke o smrtnih primerih odkritja raka pri ženskah v odstotkih.

In [150]:
kdo_umre_od_raka_z = vsi_podatki[['Death Cases Female (%)']].replace(0, '/')
print(kdo_umre_od_raka_z)

                                                   Death Cases Female (%)
Type Of Cancer                                                           
Anus                                                                  0.3
Bladder                                                               2.0
Bone and Joint                                                        0.4
Brain and Other Nervous System                                        3.6
Breast (Female)                                                      19.6
Cervix Uteri                                                          2.2
Childhood Brain and Other Nervous System (Ages ...                    0.6
Childhood Leukemia (Ages 0–19)                                        0.5
Colon and Rectum                                                     11.0
Esophagus                                                             1.4
Kidney and Renal Pelvis                                               2.2
Larynx                                

Ta tabela prikazuje podatke o povprečni starosti pri kateri oseba umre za določenim tipom raka.

In [151]:
povp_starost_smrti = vsi_podatki[['Median Age At Death']].replace(0, '/')
print(povp_starost_smrti)

                                                    Median Age At Death
Type Of Cancer                                                         
Anus                                                                 67
Bladder                                                              79
Bone and Joint                                                       66
Brain and Other Nervous System                                       66
Breast (Female)                                                      70
Cervix Uteri                                                         59
Childhood Brain and Other Nervous System (Ages ...                    9
Childhood Leukemia (Ages 0–19)                                       11
Colon and Rectum                                                     72
Esophagus                                                            70
Kidney and Renal Pelvis                                              72
Larynx                                                          

V spodaj prikazani tabeli so podatki o 5 letni stopnji preživetja izraženi v odstotkih.

In [153]:
moznost_prezivetja = vsi_podatki[['5-Year Relative Survival (%)']].replace(0, '/')
print(moznost_prezivetja)

                                                   5-Year Relative Survival (%)
Type Of Cancer                                                                 
Anus                                                                       70.4
Bladder                                                                    77.9
Bone and Joint                                                             68.9
Brain and Other Nervous System                                             33.8
Breast (Female)                                                            90.8
Cervix Uteri                                                               67.2
Childhood Brain and Other Nervous System (Ages ...                            /
Childhood Leukemia (Ages 0–19)                                                /
Colon and Rectum                                                           65.0
Esophagus                                                                  21.7
Kidney and Renal Pelvis                 

Podatke želimo predstaviti tudi grafično, zato definiramo funkcijo za generiranje grafov.

In [154]:
def generiraj_graf(naslov, vrednosti):
    ax1 = vrednosti.plot(kind='barh', legend=False, figsize=(25, 8), width=0.8, color='#4169E1')
    ax1.set_title(naslov)
    ax1.set_ylabel(' ')
    plt.savefig(mapa_grafi + naslov.replace(' ', '_') + '.jpg', format='jpeg')
    plt.close()

In [155]:
generiraj_graf('Estimated New Cases in 2023', pricakovani_novi_primeri)

Prvi graf prikazuje pričakovane nove primere v letu 2023. 
![Estimated%20New%20Cases%20in%202023.jpg](grafi/Estimated%20New%20Cases%20in%202023.jpg)

In [156]:
generiraj_graf('Estimated Deaths in 2023', pricakovani_smrtni_primeri)

Drugi graf prikazuje pričakovane smrtne primere v letu 2023.
![smrtni primeri](grafi/Estimated%20Deaths%20in%202023.jpg)

Enako naredimo še za generiranje grafov, kjer bo prikazana primerjava med šteilom moških in številom žensk, ki dobijo oziroma umrejo za rakom. Za vsak tip raka se bo posebaj izrisal graf, ki bo prikaz delež moških in delež žensk pri katerem so odkrili raka oziroma so umrli za rakom.

In [163]:
def generiraj_graf_primerjave(v1, v2, naslov):
    naslov = naslov.replace(' ', '_')
    primerjava = vsi_podatki[[v1, v2]]
    primerjava.loc[:, [v1, v2]] = primerjava[[v1, v2]].applymap(pd.to_numeric, errors='coerce')

    for index, row in primerjava.iterrows():
        values = row.values

        plt.bar(range(len(values)), values, color=['#A4D3EE', '#FFAEB9'])

        plt.ylim(bottom=0)
        plt.title(f'Type of cancer: {index}')

        plt.savefig(f'{mapa_grafi}{naslov}{index}.jpg', format='jpeg')
        plt.clf()
        plt.close()

Generiramo grafe tako, da uporabimo funkcijo.

In [164]:
generiraj_graf_primerjave('New Cases Male (%)', 'New Cases Female (%)', 'bar_plot_new_cases_')

Spodnji graf nam prikazuje primerjavo med deležem žensk in moških, ki jim diagnosticirajo raka jezika.

![rak jezika](grafi/bar_plot_new_cases_Tongue.jpg)

Generiramo še grafe za smrtne primere.

In [165]:
generiraj_graf_primerjave('Death Cases Male (%)', 'Death Cases Female (%)', 'bar_plot_death_cases_')

Zopet si lahko pogledamo primer enega izmed grafov. Spodnji graf nam prikaže delež žensk in moških, ki umrejo za rakom trebušne slinavke.

![rak trebušne slinavke](grafi/bar_plot_death_cases_Pancreas.jpg)

Na koncu bomo predstavili še kakšen delež predstavlja določen tip raka v novih primerih in pa v smrtnih primerih. Prvo kar moramo narediti je filtrirati podatke, saj niso vsi uporabni.

In [101]:
procenti = vsi_podatki[['(%) of All New Cancer Cases', '(%) of All Cancer Deaths']]
procenti = procenti[procenti['(%) of All New Cancer Cases'] != 0]

procenti.loc[:, '(%) of All New Cancer Cases'] = procenti['(%) of All New Cancer Cases'].str.replace('%', '')
procenti.loc[:, '(%) of All Cancer Deaths'] = procenti['(%) of All Cancer Deaths'].str.replace('%', '')
procenti.loc[:, ['(%) of All New Cancer Cases', '(%) of All Cancer Deaths']] = procenti[
    ['(%) of All New Cancer Cases', '(%) of All Cancer Deaths']].applymap(pd.to_numeric, errors='coerce')

procenti_novi = procenti[['(%) of All New Cancer Cases']]
procenti_smrtni = procenti[['(%) of All Cancer Deaths']]

Ker smo že napisali funkcijo za generiranje grafov, jo lahko tu samo pokličemo in vanjo ustavimo ustrezne podatke.

In [58]:
generiraj_graf('(%) of All New Cancer Cases', procenti_novi)
generiraj_graf('(%) of All Cancer Deaths', procenti_smrtni)

Prvi graf prikazuje delež določenega tipa raka v vseh novih primerih.

![delež novi primeri](grafi/(%)_of_All_New_Cancer_Cases.jpg)

Drugi graf prikazuje delež določenega tipa raka v vseh smrtnih primerih.

![delež smrtni primeri](grafi/(%)_of_All_New_Cancer_Cases.jpg)